In [2]:
import torch
import numpy as np

In [134]:
b, n, c = 2,5,2

dataset_x = torch.arange(0,20, dtype=torch.float32).reshape(b,n,c)
# print(dataset_x)

ratio = 0.6
real_n = int(n*ratio)
masked_n = n-real_n

zeros = torch.zeros(masked_n)
ones = torch.ones(real_n)
mask = torch.cat([zeros, ones], dim=0)
mask = mask[torch.randperm(n)]
print(mask)

mask = mask[None, :, None]
mask = mask.expand(b, n, c)

print(dataset_x.shape)
print(mask.shape)

# dataset_x位置编码
masked_x = dataset_x.masked_select(mask.eq(1))
masked_x = masked_x.reshape(b, real_n, c)
print(masked_x.shape)


recovery_x = torch.zeros(b,n,c) # 可学习占位符
# recovery_x位置编码
recovery_x = recovery_x.masked_scatter(mask.eq(1), masked_x)
print(recovery_x)


tensor([0., 1., 0., 1., 1.])
torch.Size([2, 5, 2])
torch.Size([2, 5, 2])
torch.Size([2, 3, 2])
tensor([[[ 0.,  0.],
         [ 2.,  3.],
         [ 0.,  0.],
         [ 6.,  7.],
         [ 8.,  9.]],

        [[ 0.,  0.],
         [12., 13.],
         [ 0.,  0.],
         [16., 17.],
         [18., 19.]]])


In [ ]:
x = torch.FloatTensor([5,6,7,8,9])
# x = x + position_embedding
print(x)

masked_x = torch.masked_select(x, mask.eq(1)) # 送入Encoder
print(masked_x)

recovery_x = torch.zeros(5)
# recovery_x = recovery_x + position_embedding

recovery_x = recovery_x.masked_scatter(mask.eq(1), masked_x)
print(recovery_x) # 送入Decoder

In [36]:
from torchvision import datasets, transforms

dataset = datasets.MNIST(
    root='./dataset', train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ]))

def collate_fn(data):

    return [
        torch.stack([x[0] for x in data]),
        torch.stack([torch.tensor(x[1]) for x in data])
    ]

dataloader = torch.utils.data.DataLoader(dataset, batch_size=17, shuffle=True, collate_fn=collate_fn)

for batch in dataloader:
    x, y = batch
    print(x.shape)
    print(y)
    print(y.dtype)
    break


torch.Size([17, 1, 28, 28])
tensor([9, 4, 1, 9, 6, 8, 9, 1, 1, 4, 9, 1, 1, 7, 3, 3, 4])
torch.int64
